In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# If the icetemperature library is not in your PYTHONPATH, you will not be able to load those functions
# Check and update here if necessary
import sys
cs_dir = '../'
if cs_dir not in sys.path:
    sys.path.append(cs_dir)

# Import the ice temperature model and relevant constants
from icetemperature.lib.numerical_model import ice_temperature
from icetemperature.lib.analytical_solutions import Robin_T, Meyer_T, Perol_T
from icetemperature.lib.ice_properties import conductivity, heat_capacity, rate_factor
from icetemperature.lib.constants import constants
const = constants()

In [ ]:
def numerical_model(adot,Ts,H,qgeo,Udef,Uslide,
                    dS=0.1,plane_strain=False,T_bulk=None,
                    long_advection=False,dTs=0.,dH=0.,dT_lat=0.,U_lat=0.,da=0.,p=12.3,ϵ_xy=0.,
                    tol=1e-8,dt=1.,A_xy='full'):

    m = ice_temperature(Ts=Ts,adot=adot,H=H,qgeo=qgeo,p=p,dTs=dTs,dH=dH,da=da,dS=dS,A_xy=A_xy,eps_xy=ϵ_xy)

    m.Udef = Udef/const.spy
    m.Uslide = Uslide/const.spy

    if long_advection:
        m.flags.append('long_advection')
    if plane_strain:
        m.flags.append('plane_strain')
    if dS > 0.:
        m.flags.append('vertical_shear')
    
    if T_bulk is None:
        m.flags.append('temp-dependent')
        m.initial_conditions()
        m.k = conductivity(m.T.copy(),m.rho)
        m.Cp = heat_capacity(m.T.copy())
    else:
        m.initial_conditions()
        m.k = conductivity(T_bulk,m.rho)*np.ones(m.nz)
        m.Cp = heat_capacity(T_bulk)*np.ones(m.nz)
        
    m.source_terms()
    m.stencil(dt*const.spy)
    m.tol=tol
    m.numerical_to_steady_state()
    
    return m

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(8,3))

ax1 = plt.subplot(133)
ax1.tick_params(labelleft=False)
plt.xlabel('Temperature ($^\circ$C)')
plt.ylim(0,1100)
plt.xlim(-28,2)
ax1.set_title('Dragon Margin')

ax2 = plt.subplot(132)
ax2.tick_params(labelleft=False)
plt.xlabel('Temperature ($^\circ$C)')
plt.ylim(0,1100)
plt.xlim(-28,2)
ax2.set_title('Kamb Ice Stream')

ax3 = plt.subplot(131)
plt.ylabel('Height Above Bed (m)')
plt.xlabel('Temperature ($^\circ$C)')
plt.ylim(0,1100)
plt.xlim(-28,2)
ax3.set_title('Siple Dome')

### Dragon ###
Ts = -26.9
H = 1000.
SMB = 66.1
adot = SMB/const.rho
qgeo = 0.2
Udef = 80.
Uslide = 200.
eps_xy = 4.75e-2
dS = 1.3e-3
print('Dragon; H:',H,'T_surface:',Ts,'Accumulation Rate:',round(adot,3))

m = ice_temperature(Ts=Ts,adot=adot,H=H,qgeo=qgeo,eps_xy=eps_xy)
TRdrag = Robin_T(m)
ax1.plot(TRdrag,m.z,'grey',zorder=1)
TM_soft = Meyer_T(m,T_bulk=None)
TM_soft += (max(m.z)-m.z)*const.rho*const.g*const.beta
Tm = const.beta*const.rho*const.g*m.H
TM_stiff = Meyer_T(m)
TM_stiff += (max(m.z)-m.z)*const.rho*const.g*const.beta
#TM_stiff = Perol_T(m)
#TM_stiff[TM_stiff>m.pmp] = m.pmp[TM_stiff>m.pmp]
ax1.fill_betweenx(m.z,TM_soft,TM_stiff,color='maroon',zorder=2,alpha=0.1)
l2, = ax1.plot(TM_soft,m.z,'maroon',zorder=2)
ax1.plot(TM_stiff,m.z,'maroon',zorder=2)
m_drag = numerical_model(adot,Ts,H,qgeo,Udef,Uslide,
                         long_advection=True,dTs=14.65e-6,dH=-2.02e-3,da=-0.48e-6,
                         ϵ_xy=eps_xy,tol=1e-9,A_xy=rate_factor(0.),plane_strain=True,dS=dS)
m_drag_visc = numerical_model(adot,Ts,H,qgeo,Udef,Uslide,
                              long_advection=True,dTs=14.65e-6,dH=-2.02e-3,da=-0.48e-6,
                             ϵ_xy=eps_xy,tol=1e-9,plane_strain=True,dS=dS)

ax1.plot(TRdrag,m.z,'k',zorder=1)
ax1.fill_betweenx(m.z,TM_soft,TM_stiff,color='maroon',zorder=2,alpha=0.1)
l2, = ax1.plot(TM_soft,m.z,'maroon',zorder=2)
ax1.plot(TM_stiff,m.z,'maroon',zorder=2)
ax1.fill_betweenx(m_drag.z,m_drag.T,m_drag_visc.T,color='steelblue',alpha=0.1)
ax1.plot(m_drag.T,m_drag.z,'steelblue')
ax1.plot(m_drag_visc.T,m_drag.z,'steelblue')


### Kamb ###
Ts = -25.52
H = 949.
SMB = 60.0
adot = SMB/const.rho
qgeo = .0663
Udef = 50.
Uslide = 250.
print('Kamb; H:',H,'T_surface:',Ts,'Accumulation Rate:',round(adot,3))

m = ice_temperature(Ts=Ts,adot=adot,H=H,qgeo=qgeo)
TRkamb = Robin_T(m)
l1, = ax2.plot(TRkamb,m.z,'grey',zorder=2)  
m_kis = numerical_model(adot,Ts,H,qgeo,Udef,Uslide,long_advection=True,dTs=6.42e-6,dH=-1.59e-3,da=-0.20e-6)
ntsteps = 100
m_kis.adot_s = np.array([m_kis.adot]*ntsteps)
m_kis.Ts_s = np.array([m_kis.Ts]*ntsteps)
dt=1.
m_kis.ts = np.arange(0,dt*ntsteps,dt)*const.spy
m_kis.Udef = 0./const.spy
m_kis.Uslide = 10./const.spy
m_kis.numerical_transient()
l4, = ax2.plot(m_kis.T_steady,m_kis.z,'steelblue')
ax2.plot(m_kis.T,m_kis.z,'navy')
    
    
### Siple ###
Ts = -24.95
H = 1005.
SMB = 105
adot = SMB/const.rho
qgeo = .0653
print('Siple; H:',H,'T_surface:',Ts,'Accumulation Rate:',round(adot,3))

m = ice_temperature(Ts=Ts,adot=adot,H=H,qgeo=qgeo)
TRsiple = Robin_T(m)
ax3.plot(TRsiple,m.z,'k',zorder=1)
m_siple = numerical_model(adot,Ts,H,qgeo,0.,0.)
ax3.plot(m_siple.T,m_siple.z,'steelblue')
ax3.legend([l1,l2,l4],['Robin (1955)','Meyer (2018)','Weertman\n(1968)'])

plt.tight_layout()